# Aggregation

In [1]:
import pandas as pd
import numpy as np
import glob
import os

In [2]:
# Path is relative to current directory
# Import all CSV files
path = '/home/miles/Dropbox/iCAP_Project/Results/Analysis'
file_names = glob.glob(path + '/[0-9]*.csv')

In [3]:
file_names

['/home/miles/Dropbox/iCAP_Project/Results/Analysis/20170206_centhud_rec.csv',
 '/home/miles/Dropbox/iCAP_Project/Results/Analysis/20170206_pseg_rec.csv',
 '/home/miles/Dropbox/iCAP_Project/Results/Analysis/20170206_peco_rec.csv',
 '/home/miles/Dropbox/iCAP_Project/Results/Analysis/20170206_ppl_rec.csv',
 '/home/miles/Dropbox/iCAP_Project/Results/Analysis/20170206_coned_rec.csv']

In [4]:
# Load all CSV files into memory
# Concatenate into single DataFrame
df = pd.DataFrame()
list_ = []
for file_ in file_names:
    df = pd.read_csv(file_, index_col=None, header=0)
    list_.append(df)
df = pd.concat(list_)

In [5]:
temp = df.copy()

In [6]:
grp = temp.groupby('PremiseId')

mad = grp['RecipeICap'].transform(lambda x: x.mad())
pct = grp['RecipeICap'].transform(lambda x: x.pct_change() * 100)
#prem_min = grp[['RecipeICap', 'Year']].transfrom(lambda x: x.ix[x.RecipeICap.idxmax()])

/home/miles/anaconda3/lib/python3.5/site-packages/pandas/core/generic.py:5374: RuntimeWarning: invalid value encountered in absolute
  return np.abs(demeaned).mean(axis=axis, skipna=skipna)


In [7]:
temp['TagVariability'] = mad
temp['PercentChange'] = pct
#temp.fillna(value='null', inplace=True)
print(temp.shape)
temp.dropna(subset=['HistoricalICap'])
temp.fillna(value='null', inplace=True)

temp = temp.groupby('PremiseId').filter(lambda x: len(x) > 1)

(52515, 13)


In [8]:
temp['Year'] = pd.to_numeric(temp['Year'], downcast='float', errors='coerce')
temp = temp[temp['RecipeICap'] != 'null']

/home/miles/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [9]:
container = {}
for iso_util_prem_year, record in temp.groupby(['ISO', 'UtilityId', 'PremiseId', 'Year']):
    iso, utility, premise, year = iso_util_prem_year

    year = str(int(year))
    
    # iso level
    if iso not in container.keys():
        container[iso] = {}
    iso_d = container[iso]
    
    # utility 
    if utility not in iso_d.keys():
        iso_d[utility] = {'aggregate':{}, 'records':{}}
    util_agg_d = iso_d[utility]['aggregate']
    util_rec_d = iso_d[utility]['records']
    

    # update utility meta data
    if year not in util_agg_d.keys():
        util_agg_d[year] = {'min':0., 'max':0., 'count':0., 'total':0}

    util_year_d = util_agg_d[year]
    
    recipe_value = record.RecipeICap.values[0]
    util_year_d['min'] = min([util_year_d['min'], recipe_value])
    util_year_d['max'] = max([util_year_d['max'], recipe_value])
    util_year_d['total'] += recipe_value
    util_year_d['count'] += 1

    # premise
    if premise not in util_rec_d.keys():
        util_rec_d[premise] = {'metadata':{'min':(0, 1000000), 'max':(0, 0.), 'count':0., 'total':0}}
    prem_d = util_rec_d[premise]
    prem_agg_d = util_rec_d[premise]['metadata']
    
    meta_cols = ['ISO', 'MeterType', 'RateClass', 'Strata', 'UtilityId', 'PremiseId',
                'TagVariability']
    prem_meta_d = record[meta_cols].to_dict(orient='records')[0]
    
    prem_agg_d['min'] = min([prem_agg_d['min'], (year, recipe_value)], key=lambda tup: tup[1])
    prem_agg_d['max'] = max([prem_agg_d['max'], (year, recipe_value)], key=lambda tup: tup[1])
    prem_agg_d['total'] += recipe_value
    prem_agg_d['count'] += 1

   
    for k, v in prem_meta_d.items():
        if k not in prem_agg_d.keys():
            prem_agg_d[k] = v
            
    
    
    # year
    if year not in prem_d.keys():
        prem_d[year] = {}
    year_d = prem_d[year]
    
    
    
    for k,v in record.drop(meta_cols, axis=1).to_dict(orient='records')[0].items():
        year_d[k] = v
    
    
                    

In [10]:
container

{'PJM': {'CENTHUD': {'aggregate': {'2015': {'count': 319.0,
     'max': 30981.0452,
     'min': 0.0,
     'total': 227527.58539999995},
    '2016': {'count': 322.0,
     'max': 30273.8021,
     'min': 0.0,
     'total': 269518.9418000004}},
   'records': {1107096000: {'2015': {'HistVar': 61.0782,
      'HistoricalICap': 1.33,
      'PercentChange': 'null',
      'RecipeICap': 0.5177,
      'RunDate': '2017-02-06_11:16',
      'Year': 2015.0},
     '2016': {'HistVar': 42.2321,
      'HistoricalICap': 0.76,
      'PercentChange': 108.80819007146995,
      'RecipeICap': 1.081,
      'RunDate': '2017-02-06_11:16',
      'Year': 2016.0},
     'metadata': {'ISO': 'PJM',
      'MeterType': 'CON',
      'PremiseId': 1107096000,
      'RateClass': 2,
      'Strata': 'ND1',
      'TagVariability': 0.28164999999999996,
      'UtilityId': 'CENTHUD',
      'count': 2.0,
      'max': ('2016', 1.081),
      'min': ('2015', 0.5177),
      'total': 1.5987}},
    1206260000: {'2015': {'HistVar': 39.0988

In [11]:
import json
with open('ppl_premise_explorer.json', 'w') as f:
    json.dump(container, f, indent=4, separators=(',',': '))